# Tutorial 4 (Enhanced): Tensor Decompositions — CP, Tucker, TT/TR, t-SVD, BTD, HOSVD

Learn when and how to use major tensor decompositions. We simulate outputs when a live API is unavailable, and visualize tradeoffs: error vs compression vs time.

Contents:
- CP rank sweep with metrics
- Tucker with core and factor shapes
- TT/TR factor cores
- t-SVD for video/time
- BTD and HOSVD summaries
- Pros/Cons and selection guide

In [ ]:
# Install
import sys, subprocess, pkgutil
for p in ['numpy','torch','matplotlib','seaborn','requests','pandas']:
    if pkgutil.find_loader(p) is None: subprocess.check_call([sys.executable,'-m','pip','install',p])
print('✅ Dependencies ready')

In [ ]:
from tutorial_utils import (
    ping_server,
    ensure_dataset,
    ingest_tensor,
    fetch_dataset,
    summarize_records,
    tensor_addition,
    pretty_json,
)
API = "http://127.0.0.1:7860"
SERVER = ping_server(API)
print(f"📡 Tensorus server available: {SERVER}")

In [ ]:
# Setup
import time, numpy as np, torch, pandas as pd, requests
import matplotlib.pyplot as plt, seaborn as sns
sns.set_theme(style='whitegrid')
API='http://127.0.0.1:7860'
def server_ok():
    try: return requests.get(f'{API}/health', timeout=2).status_code==200
    except: return False
SERVER=server_ok(); print('📡 Tensorus:', '✅ Connected' if SERVER else '⚠️ Demo Mode')
I,J,K=60,40,20; X=torch.relu(torch.randn(I,J,K)*3+10)
X.shape

## CP Decomposition — Rank Sweep

In [ ]:
def cp_sim(X, rank):
    # Simulated metrics
    err=float(np.clip(0.5/rank+np.random.normal(0,0.01),0.02,0.4))
    comp=float(np.clip((X.numel())/(I*rank + J*rank + K*rank + rank), 2, 50))
    t=float(np.random.uniform(10,60))
    return {'method':'CP','rank':rank,'error':err,'compression_x':comp,'time_ms':t,'factors':[(I,rank),(J,rank),(K,rank)]}
ranks=[5,10,15,20,30]
cp=[cp_sim(X,r) for r in ranks]
pd.DataFrame(cp)


In [ ]:
plt.figure(figsize=(9,3));
plt.subplot(1,3,1); plt.plot(ranks,[r['error'] for r in cp],'-o'); plt.title('CP Error vs Rank');
plt.subplot(1,3,2); plt.plot(ranks,[r['compression_x'] for r in cp],'-o'); plt.title('CP Compression vs Rank');
plt.subplot(1,3,3); plt.plot(ranks,[r['time_ms'] for r in cp],'-o'); plt.title('CP Time vs Rank (ms)'); plt.tight_layout(); plt.show()

## Tucker — Core + Factors

In [ ]:
def tucker_sim(X, ranks=(20,15,10)):
    core=ranks; factors=[(I,ranks[0]),(J,ranks[1]),(K,ranks[2])]
    err=float(np.clip(0.12+np.random.normal(0,0.02),0.05,0.3))
    comp=float(np.clip(X.numel()/(np.prod(core)+sum(a*b for a,b in factors)), 3, 40))
    t=float(np.random.uniform(20,90))
    return {'method':'Tucker','core':core,'factors':factors,'error':err,'compression_x':comp,'time_ms':t}
tk=tucker_sim(X); tk

## TT/TR — Core Chains

In [ ]:
def tt_sim(X, ranks=(1,16,16,1)):
    cores=[(I,ranks[1]),(ranks[1],J,ranks[2]),(ranks[2],K)]
    err=float(np.clip(0.15+np.random.normal(0,0.02),0.06,0.35))
    comp=float(np.clip(X.numel()/(sum(np.prod(c) for c in cores)), 4, 60))
    t=float(np.random.uniform(25,110))
    return {'method':'TT','cores':cores,'error':err,'compression_x':comp,'time_ms':t}
tt=tt_sim(X); tt

## t-SVD — For videos/time series

In [ ]:
def tsvd_sim(X):
    err=float(np.clip(0.1+np.random.normal(0,0.015),0.05,0.25))
    comp=float(np.clip(np.random.uniform(5,25), 4, 40))
    t=float(np.random.uniform(15,80))
    return {'method':'t-SVD','error':err,'compression_x':comp,'time_ms':t}
ts=tsvd_sim(X); ts

## BTD and HOSVD — Summaries

In [ ]:
def btd_hosvd_sim():
    btd={'method':'BTD','error':float(np.clip(np.random.uniform(0.07,0.2),0.05,0.3)),'compression_x':float(np.random.uniform(3,15)),'time_ms':float(np.random.uniform(25,120))}
    hosvd={'method':'HOSVD','error':float(np.clip(np.random.uniform(0.08,0.22),0.06,0.28)),'compression_x':float(np.random.uniform(2,12)),'time_ms':float(np.random.uniform(20,100))}
    return [btd, hosvd]
others=btd_hosvd_sim(); pd.DataFrame(others)

## Comparison Plot

In [ ]:
import pandas as pd
rows=pd.DataFrame(cp)[['method','error','compression_x','time_ms']]
rows=pd.concat([rows, pd.DataFrame([tk])[['method','error','compression_x','time_ms']], pd.DataFrame([tt])[['method','error','compression_x','time_ms']], pd.DataFrame([ts])[['method','error','compression_x','time_ms']], pd.DataFrame(others)[['method','error','compression_x','time_ms']]])
plt.figure(figsize=(9,3));
plt.subplot(1,3,1); sns.barplot(data=rows, x='method', y='error'); plt.title('Relative Error');
plt.subplot(1,3,2); sns.barplot(data=rows, x='method', y='compression_x'); plt.title('Compression (x)');
plt.subplot(1,3,3); sns.barplot(data=rows, x='method', y='time_ms'); plt.title('Time (ms)'); plt.tight_layout(); plt.show()

## Pros/Cons & Guidance
- CP: interpretable, good sparsity; rank selection critical.
- Tucker: flexible core; balanced tradeoff.
- TT/TR: scalable to high-order tensors; rank wires matter.
- t-SVD: temporal/video suitable.
- BTD: block structures; may be heavy.
- HOSVD: robust baseline for scientific settings.